# [Step 2] ทำการนำเข้า module ที่จำเป็นต้องใช้ในการทำงานบน Environment (For Model CNN)

In [ ]:
import tensorflow as tf
import tensorflow.keras as ks
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import random

In [ ]:
#ตรวจสอบและตั้งค่า Directory เพื่อนำเข้าข้อมูลต้นทางที่เป็น .npy format
os.chdir(r"C:\Users\std\Desktop\Dataset\CNN Method\SplitIMG_3x3\Output")

#Load array file to spilt data
HKT_IMG_arr = np.load('HKT_CNN_3x3_09022018.npy')
Label_IMG_arr = np.load('Label_CNN_3x3_09022018.npy')

### [Step 2.1] ทำการสร้าง Function ในการจำแนกข้อมูลเพื่อนำไป Train บนตัว Model

In [ ]:
#Define the function split data
def train_test_spilt(HKT_Arr, Labels_Arr, trainProp=0.6):
    datasize = HKT_Arr.shape[0]
    sliceindex = int(datasize*trainProp)
    randindex = np.arange(datasize)
    random.shuffle(randindex)
    X_train = HKT_Arr[[randindex[:sliceindex]], :, :, :][0]
    X_test = HKT_Arr[[randindex[sliceindex:]], :, :, :][0]
    Y_train = Labels_Arr[randindex[:sliceindex]]
    Y_test = Labels_Arr[randindex[sliceindex:]]
    return (X_train, X_test, Y_train, Y_test)

In [ ]:
#Spilt the data // ทำการแยกข้อมูลเป็น 2 ประเภท คือ Train = ข้อมูลฝึกฝนแบบจำลอง | Test = ข้อมูลทดสอบแบบจำลอง
X_train, X_test, Y_train, Y_test = train_test_spilt(HKT_IMG_arr, Label_IMG_arr)

# Transpose the features to channel last format // จัดลำดับตัวข้อมูลใหม่
train_x = tf.transpose(X_train, [0, 2, 3, 1])
test_x = tf.transpose(X_test, [0, 2, 3, 1])
print('Reshaped features:', train_x.shape, test_x.shape)
_, rowSize, colSize, nBands = train_x.shape

### [Step 2.2] สร้างแบบจำลอง CNN (Model)

In [ ]:
# Create a model
model = ks.Sequential()
model.add(Conv2D(50, kernel_size=1, padding='valid', activation='relu', input_shape=(rowSize, colSize, nBands)))
model.add(Conv2D(75, kernel_size=1, padding='valid', activation='relu'))
model.add(Dropout(0.25))
model.add(Conv2D(100, kernel_size=1, padding='valid', activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(6, activation='softmax'))

#แสดง parameter เพื่อดูขนาดและจำนวน node ที่ Modelใช้ฝึกฝนและจำลอง
model.summary()

### [Step 2.3] ทำการ Train Model และทดสอบการจำแนกเพื่อทำนายผลที่ได้ (จำแนกการใช้ประโยชน์ที่ดิน)

In [ ]:
# Run the model
batch_size=2
model.compile(loss='sparse_categorical_crossentropy', optimizer= 'rmsprop',metrics=['accuracy'])
model.fit(train_x, Y_train, batch_size=batch_size, epochs=50, verbose=1, validation_data=(test_x, Y_test))

#Predicted data from CNM model
yTestPredicted = model.predict(test_x)
yTestPredicted = np.argmax(yTestPredicted, axis=1)

### [Step 2.4] แสดงผลลัพธ์และรายงานผลประสิทธิภาพของตัว Model ที่ได้ทำการ Train

In [ ]:
# Calculate and display the error metrics
cMatrix = confusion_matrix(Y_test, yTestPredicted)
pScore = precision_score(Y_test, yTestPredicted, average='micro')
rScore = recall_score(Y_test, yTestPredicted, average='micro')
f1Score = f1_score(Y_test, yTestPredicted, average='micro')

print("Confusion matrix: for 24 nodes\n", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f, F1-Score: %.3f" % (pScore, rScore, f1Score))
print("="*35)
from sklearn.metrics import classification_report, accuracy_score

#Classification Report
print('Classification Report: \n%s' % classification_report(Y_test, yTestPredicted))
print('Classification accuracy" %s' % accuracy_score(Y_test, yTestPredicted))